In [10]:
import dask, concurrent.futures, time, warnings, os, re, pickle
from osgeo import gdal
import requests as r
import panel as pn
pn.extension()
import param as pm
import pandas as pd
from collections import OrderedDict as odict
import numpy as np
from dask.distributed import LocalCluster, Client
import xarray as xr
import hvplot.pandas
import hvplot.xarray
import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen
from xml.etree.ElementTree import parse,fromstring
from affine import Affine
from pandas import to_datetime
import jinja2 as jj2
from rasterio.crs import CRS
from tempfile import NamedTemporaryFile
from datetime import datetime
from netrc import netrc
from subprocess import Popen
from pyproj import Proj
from src.hls_funcs.masks import mask_hls
from src.hls_funcs.predict import pred_cov, pred_bm, pred_bm_se, pred_bm_thresh
import cartopy.crs as ccrs
from bokeh.models.formatters import PrintfTickFormatter
import stackstac
from subprocess import Popen, DEVNULL, STDOUT
from getpass import getpass
from sys import platform

In [11]:
# Create a LUT dict including the HLS product bands mapped to names
lut = {'HLSS30':
       {'B01': 'COASTAL-AEROSOL',
        'B02': 'BLUE', 
        'B03': 'GREEN', 
        'B04': 'RED', 
        'B05': 'RED-EDGE1',
        'B06': 'RED-EDGE2', 
        'B07': 'RED-EDGE3',
        'B08': 'NIR-Broad',
        'B8A': 'NIR1', 
        'B09': 'WATER-VAPOR',
        'B10': 'CIRRUS',
        'B11': 'SWIR1', 
        'B12': 'SWIR2', 
        'Fmask': 'FMASK'},
       'HLSL30': 
       {'B01': 'COASTAL-AEROSOL',
        'B02': 'BLUE', 
        'B03': 'GREEN', 
        'B04': 'RED', 
        'B05': 'NIR1',
        'B06': 'SWIR1',
        'B07': 'SWIR2', 
        'B09': 'CIRRUS', 
        'B10': 'TIR1', 
        'B11': 'TIR2', 
        'Fmask': 'FMASK'}}

# List of all available/acceptable band names
all_bands = ['ALL', 'COASTAL-AEROSOL', 'BLUE', 'GREEN', 'RED', 'RED-EDGE1', 'RED-EDGE2', 'RED-EDGE3', 
             'NIR1', 'SWIR1', 'SWIR2', 'CIRRUS', 'TIR1', 'TIR2', 'WATER-VAPOR', 'FMASK']

needed_bands = ['BLUE', 'GREEN', 'RED', 'NIR1', 'SWIR1', 'SWIR2', 'FMASK']

In [12]:
def NASA_CMR_STAC(hls_data, aws):
    stac = 'https://cmr.earthdata.nasa.gov/stac/' # CMR-STAC API Endpoint
    stac_response = r.get(stac).json()            # Call the STAC API endpoint
    stac_lp = [s for s in stac_response['links'] if 'LP' in s['title']]  # Search for only LP-specific catalogs

    # LPCLOUD is the STAC catalog we will be using and exploring today
    lp_cloud = r.get([s for s in stac_lp if s['title'] == 'LPCLOUD'][0]['href']).json()
    lp_links = lp_cloud['links']
    lp_search = [l['href'] for l in lp_links if l['rel'] == 'search'][0]  # Define the search endpoint
    lim = 100
    search_query = f"{lp_search}?&limit={lim}"    # Add in a limit parameter to retrieve 100 items at a time.
    bbox_num=[-104.79107047,   40.78311181, -104.67687336,   40.87008987]
    bbox = f'{bbox_num[0]},{bbox_num[1]},{bbox_num[2]},{bbox_num[3]}'  # Defined from ROI bounds
    search_query2 = f"{search_query}&bbox={bbox}"                                                  # Add bbox to query
    date_time = hls_data['date_range'][0]+'/'+hls_data['date_range'][1]  # Define start time period / end time period
    search_query3 = f"{search_query2}&datetime={date_time}"  # Add to query that already includes bbox
    collections = r.get(search_query3).json()['features']    
    hls_collections = [c for c in collections if 'HLS' in c['collection']]
    s30_items = [h for h in hls_collections if h['collection'] == 'HLSS30.v1.5']  # Grab HLSS30 collection
    l30_items = [h for h in hls_collections if h['collection'] == 'HLSL30.v1.5']  # Grab HLSL30 collection
    
    if aws:
        for stac in s30_items:
            for band in stac['assets']:
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://lpdaac.earthdata.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
                
        for stac in l30_items:
            for band in stac['assets']:
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://lpdaac.earthdata.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
                stac['assets'][band]['href'] = stac['assets'][band]['href'].replace('https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected', 
                                                                                    '/vsis3/lp-prod-protected')
    return {'S30': s30_items,
            'L30': l30_items}

def setup_netrc(creds,aws):
    urs = 'urs.earthdata.nasa.gov' 
    try:
        netrcDir = os.path.expanduser("~/.netrc")
        netrc(netrcDir).authenticators(urs)[0]
        del netrcDir

    # Below, create a netrc file and prompt user for NASA Earthdata Login Username and Password
    except FileNotFoundError:
        homeDir = os.path.expanduser("~")
        Popen('touch {0}.netrc | chmod og-rw {0}.netrc | echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
        Popen('echo login {} >> {}.netrc'.format(creds[0], homeDir + os.sep), shell=True)
        Popen('echo password {} >> {}.netrc'.format(creds[1], homeDir + os.sep), shell=True)
        del homeDir

    # Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
    except TypeError:
        homeDir = os.path.expanduser("~")
        Popen('echo machine {1} >> {0}.netrc'.format(homeDir + os.sep, urs), shell=True)
        Popen('echo login {} >> {}.netrc'.format(creds[0], homeDir + os.sep), shell=True)
        Popen('echo password {} >> {}.netrc'.format(creds[1], homeDir + os.sep), shell=True)
        del homeDir
    del urs
    if aws:
        return(r.get('https://lpdaac.earthdata.nasa.gov/s3credentials').json())
    else:
        return('')

def build_xr(stac_dict):
    try:
        s30_stack = stackstac.stack(stac_dict['S30'], epsg=32613, resolution=30, assets=[i for i in lut['HLSS30'] if lut['HLSS30'][i] in needed_bands],
                                   chunksize=(4000, 4000)) #, reader=stackstac.reader_protocol.FakeReader
        s30_stack['band'] = [lut['HLSS30'][b] for b in s30_stack['band'].values]
        s30_stack['time'] = [datetime.fromtimestamp(t) for t in s30_stack.time.astype('int').values//1000000000]
        s30_stack = s30_stack.to_dataset(dim='band').reset_coords(['end_datetime', 'start_datetime'], drop=True)
    except ValueError:
        s30_stack = None
    try:
        l30_stack = stackstac.stack(stac_dict['L30'], epsg=32613, resolution=30, assets=[i for i in lut['HLSL30'] if lut['HLSL30'][i] in needed_bands],
                                   chunksize=(4000, 4000))
        l30_stack['band'] = [lut['HLSL30'][b] for b in l30_stack['band'].values]
        l30_stack['time'] = [datetime.fromtimestamp(t) for t in l30_stack.time.astype('int').values//1000000000]
        l30_stack = l30_stack.to_dataset(dim='band').reset_coords(['name', 'end_datetime', 'start_datetime'], drop=True)
    except ValueError:
        l30_stack = None
    if s30_stack is not None and l30_stack is not None:
        hls_stack = xr.concat([s30_stack, l30_stack], dim='time')
    elif s30_stack is not None:
        hls_stack = s30_stack
    elif l30_stack is not None:
        hls_stack = l30_stack
    else:
        print('No data found for date range')
    return hls_stack.chunk({'time': 1, 'y': -1, 'x': -1})
    
def get_hls(creds, hls_data={}, aws=False):
    #Seteup creds
    
    s3_cred = setup_netrc(creds,aws=aws)
    #define gdalenv
    if aws:
        
        env = dict(GDAL_DISABLE_READDIR_ON_OPEN='FALSE', 
                   #AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   CPL_VSIL_CURL_ALLOWED_EXTENSIONS='TIF',
                   GDAL_HTTP_UNSAFESSL='YES',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'),
                   AWS_REGION='us-west-2',
                   AWS_SECRET_ACCESS_KEY=s3_cred['secretAccessKey'],
                   AWS_ACCESS_KEY_ID=s3_cred['accessKeyId'],
                   AWS_SESSION_TOKEN=s3_cred['sessionToken'])
    else:
        env = dict(GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR', 
                   AWS_NO_SIGN_REQUEST='YES',
                   GDAL_MAX_RAW_BLOCK_CACHE_SIZE='200000000',
                   GDAL_SWATH_SIZE='200000000',
                   VSI_CURL_CACHE_SIZE='200000000',
                   GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                   GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))


    os.environ.update(env)
    
    catalog = NASA_CMR_STAC(hls_data, aws)
    da  = build_xr(catalog)
    return da

In [ ]:
data_dict = {'date_range': [str(datetime(2021, 1, 1).date()), str(datetime.now().date())]}
tmp_data = get_hls(['spkearney', '1mrChamu'], hls_data=data_dict, aws=True)
bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
da = tmp_data.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))]
da_mask = mask_hls(da['FMASK'])
da = da.where(da_mask == 0)
da_bm = pred_bm(da, bm_mod)
da_bm = da_bm.where(da_bm > 0)
da_se = pred_bm_se(da, bm_mod)
da_se = da_se.where(da_bm > 0)
t0 = time.time()
print(da_bm.isel(time=1).values)
t1 = time.time()
print(t1-t0)

In [ ]:
cluster = LocalCluster(threads_per_worker=1)
cl = Client(cluster)
cl

In [ ]:
data_dict = {'date_range': [str(datetime(2021, 1, 15).date()), str(datetime.now().date())]}
tmp_data = get_hls(['spkearney', '1mrChamu'], hls_data=data_dict, aws=True)
bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
da = tmp_data.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))]
da_mask = mask_hls(da['FMASK'])
da = da.where(da_mask == 0)
da_bm = pred_bm(da, bm_mod)
da_bm = da_bm.where(da_bm > 0)
da_se = pred_bm_se(da, bm_mod)
da_se = da_se.where(da_bm > 0)
t0 = time.time()
print(da_bm.isel(time=1).values)
t1 = time.time()
print(t1-t0)

In [ ]:
from cartopy import crs
import geoviews as gv
import holoviews as hv
from copy import deepcopy
from src.objects.charts import gauge_obj
from holoviews import streams
import affine
from src.hls_funcs.masks import shp2mask

dat = da
dat['time'] = dat.time.dt.floor("D")
dat = dat.rename(dict(time='date'))
bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
cov_mod = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))

In [ ]:
class getData(pm.Parameterized):
    bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
    cov_mod = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))
    action = pm.Action(lambda x: x.param.trigger('action'), label='Load Data and Run Analysis')
    get_stats = pm.Action(lambda y: y.param.trigger('get_stats'), label='Calculate stats')
    username_input = pn.widgets.PasswordInput(name='NASA Earthdata Login', placeholder='Enter Username...')
    password_input = pn.widgets.PasswordInput(name='', placeholder='Enter Password...')
    year_picker = pn.widgets.Select(name='Year', options=[2021], value=2021)
    thresh_picker = pn.widgets.IntSlider(name='Threshold', start=200, end=2000, step=25, value=500,
                                 format=PrintfTickFormatter(format='%d kg/ha'))
    
    data_dict = {'date_range': [str(datetime(year_picker.value, 1, 1).date()),
                                str(datetime(year_picker.value, 12, 31).date())]}
 
    datCRS = crs.UTM(13)
    mapCRS = crs.GOOGLE_MERCATOR
    datProj = Proj(datCRS.proj4_init)
    mapProj = Proj(mapCRS.proj4_init)
    map_args = dict(crs=datCRS, rasterize=True, project=True, dynamic=True)
    base_opts = dict(projection=mapCRS, backend='bokeh', xticks=None, yticks=None, width=900, height=700,
                         padding=0, active_tools=['pan', 'wheel_zoom'], toolbar='left')
    map_opts = dict(projection=mapCRS, responsive=False, xticks=None, yticks=None, width=900, height=700,
                     padding=0, tools=['pan', 'wheel_zoom', 'box_zoom'],
                     active_tools=['pan', 'wheel_zoom'], toolbar='left')
    
    poly_opts = dict(fill_color=['', ''], fill_alpha=[0.0, 0.0], line_color=['#1b9e77', '#d95f02'],
                 line_width=[3, 3])
    
    bg_col='#ffffff'

    css = '''
    .bk.box1 {
      background: #ffffff;
      border-radius: 5px;
      border: 1px black solid;
    }
    '''
    
    date = pn.widgets.DatePicker(name='Calendar', width=200)
      
    box_ctr_dat = [522435.0, 4519550.5]
    box_ctr_coord = datProj(box_ctr_dat[0], box_ctr_dat[1], inverse=True)
    box_ctr_map = mapProj(box_ctr_coord[0], box_ctr_coord[1], inverse=False)
    
    tiles = gv.tile_sources.EsriImagery.opts(**base_opts, level='glyph',
                                                                   xlim=(box_ctr_map[0] - 5000,
                                                                         box_ctr_map[0] + 5000),
                                                                   ylim=(box_ctr_map[1] - 5000,
                                                                         box_ctr_map[1] + 5000))
    labels = gv.tile_sources.EsriReference.opts(**base_opts, level='overlay')
       
    basemap = gv.tile_sources.EsriImagery.opts(projection=mapCRS, backend='bokeh', level='glyph')
    
    base_rng = hv.streams.RangeXY(source=tiles)
    
    polys = hv.Polygons([])

    poly_stream = streams.PolyDraw(source=polys, drag=True, num_objects=2,
                                    show_vertices=True, styles=poly_opts)
    edit_stream = streams.PolyEdit(source=polys, shared=True)
        
    def __init__(self, **params):
        super(getData, self).__init__(**params)
        self.da = None
        self.da_cov = None
        self.da_bm = None
        self.da_bm_se = None
        self.da_thresh = None
        self.all_maps = None
        self.main = pn.pane.panel(self.tiles * self.labels, sizing_mode='stretch_both')

        self.cov_map = None
        self.bm_map = None
        self.thresh_map = None
        
        self.cov_stats = None
        self.bm_stats = 0
        self.thresh_stats = None
        
        self.view = self._create_view()
        
    
    @pm.depends('action')
    def proc_data(self):
        message = 'Not yet launched'
        if self.username_input.value != '':
            try:
                tmp_data = get_hls([self.username_input.value,self.password_input.value], 
                                   hls_data=self.data_dict, aws=True)
                #self.da = tmp_data.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))]
                message = self.base_rng.x_range
                coord_rng = self.mapProj(self.base_rng.x_range, self.base_rng.y_range, inverse=True)
                message = coord_rng
                dat_rng = np.round(self.datProj(coord_rng[0], coord_rng[1], inverse=False), 0)
                message = dat_rng
                self.da = tmp_data.loc[dict(x=slice(*dat_rng[0]), y=slice(*dat_rng[1][::-1]))]
                self.da['time'] = self.da.time.dt.floor("D")
                self.da = self.da.rename(dict(time='date'))
                da_mask = mask_hls(self.da['FMASK'])
                self.da = self.da.where(da_mask == 0)
                self.date.enabled_dates = [pd.Timestamp(x).to_pydatetime().date() for x in self.da['date'].values]
                message = 'Success!' + str(datetime.now())
                return message
            except:
                return message + ': App Failure'
        else:
            return message
    
    #@pm.depends('get_stats')
    #def update_polys(self):
    #    if self.poly_stream.data is not None:
    #        self.polys = self.poly_stream.element
    #        self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
    #                       show_vertices=True, styles=self.poly_opts)
    #        self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
    #    return self.polys
    
    @pm.depends('action', 'date.param', watch=True)
    def create_maps(self):
        if self.da is not None:
            if self.date.value is not None:
                date_idx = self.date.enabled_dates.index(self.date.value)
                da_sel = self.da.isel(date=date_idx).persist()
            else:
                da_sel = self.da.isel(date=-1).persist()
            da_bm = pred_bm(da_sel, self.bm_mod)
            da_bm = da_bm.where(da_bm > 0)
            da_bm.name = 'Biomass'
            da_cov = pred_cov(da_sel, self.cov_mod)
            da_cov = da_cov[['SD', 'GREEN', 'BARE']].to_array(dim='type')
            da_cov = da_cov.where((da_cov < 1.0) | (da_cov.isnull()), 1.0)
            da_cov.name = 'Cover'
            da_bm_se = pred_bm_se(da_sel, bm_mod)
            da_bm_se = da_bm_se.where(da_bm.notnull())
            da_thresh = pred_bm_thresh(da_bm, da_bm_se, self.thresh_picker.value)
            da_thresh.name = 'Threshold'
            
            #self.basemap = self.basemap.opts(xlim=(self.base_rng.x_range[0],
            #                                       self.base_rng.x_range[1]),
            #                                 ylim=(self.base_rng.y_range[0],
            #                                       self.base_rng.y_range[1]))
            self.cov_map = da_cov.hvplot.rgb(x='x', y='y', bands='type',
                                        **self.map_args).opts(**self.map_opts)
            self.bm_map = da_bm.hvplot(x='x', y='y',
                                  cmap='Viridis', clim=(100, 1000), colorbar=False,
                                  **self.map_args).opts(**self.map_opts)
            self.thresh_map = da_thresh.hvplot(x='x', y='y',
                                          cmap='YlOrRd', clim=(0.05, 0.95), colorbar=False,
                                          **self.map_args).opts(**self.map_opts)

            self.da_cov = da_cov
            self.da_bm = da_bm
            self.da_bm_se = da_bm_se
            self.da_thresh = da_thresh
            
            self.all_maps = pn.Tabs(#('Imagery', self.basemap * self.polys),
                           ('Cover', self.basemap * self.cov_map * self.polys),
                           ('Biomass', self.basemap * self.bm_map * self.polys),
                           ('Threshold', self.basemap * self.thresh_map * self.polys))
            
            #self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
            #                        show_vertices=True, styles=self.poly_opts)
        
        return self.all_maps
    
    def roi_stat(data):
        if not data or not any(len(d) for d in data.values()):
            return hv.Histogram()
        
        hists = {}
        for i, (xs, ys) in enumerate(zip(data['xs'], data['ys'])):
        r, c = polygon(xs, ys)
        selection = ds.select(x=np.unique(r), y=np.unique(c))
        hists[i] = hv.Histogram(selection.aggregate(np.mean))
        return hv.NdOverlay(hists)
    
    @pm.depends('get_stats')
    def show_stats(self):
        self.bm_stats = self.bm_stats + 1
        #if self.poly_stream.data is not None:
        #    self.bm_stats = 'clicked1'
        #    thresh_list = []
        #    bm_list = []
        #    cov_list = []
        #    ts_yr_list = []
        #    ts_avg_list = []
        #    self.bm_stats = 'clicked2'
        #    r, c = polygon([-1.0 * x for x in self.poly_stream.data['xs'][0]], self.poly_stream.data['ys'][0])
        #    r = r * -1.0
        #    self.bm_stats = self.bm_map[:,c][r,:].data.mean().Biomass.values        
        return pn.Row(self.bm_stats)
    
    @pm.depends('action', 'date.param')
    def _update_main(self):
        if self.da is not None:
            self.main = self.all_maps
        else:
            self.main.object = self.tiles * self.labels
        return self.main
    
    def _create_view(self):
        dmap = hv.DynamicMap(self.roi_curves, streams=[self.poly_stream])
        layout = pn.Row(pn.Column(self.username_input,
              self.password_input,
              pn.Param(self.param,
                       widgets={'action': pn.widgets.Button(name='Get data',
                                                               width=200),
                               'get_stats': pn.widgets.Button(name='Calculate stats',
                                                               width=200)},
                      show_name=False),
             self.proc_data,
             self.date), self._update_main,
                        self.show_stats)
        return layout

In [ ]:
data_step = getData()

In [ ]:
layout = pn.Row(pn.Column(data_step.username_input,
                          data_step.password_input,
                          pn.Param(data_step.param,
                                   widgets={'action': pn.widgets.Button(name='Get data',
                                                                           width=200),
                                           'get_stats': pn.widgets.Button(name='Calculate stats',
                                                                           width=200)},
                                  show_name=False),
                         data_step.proc_data,
                         data_step.date), data_step.create_maps,
               data_step.show_stats)
layout.servable()

In [ ]:
data_step.poly_stream.data

In [ ]:
data_step.bg_col='#ffffff'

data_step.css = '''
.bk.box1 {
  background: #ffffff;
  border-radius: 5px;
  border: 1px black solid;
}
'''
thresh_list = []
bm_list = []
cov_list = []
ts_yr_list = []
ts_avg_list = []
for idx, ps_c in enumerate(data_step.poly_stream.data['line_color']):
    print('clicked2')
    xs_map, ys_map = data_step.mapProj(data_step.poly_stream.data['xs'][idx],
                                  data_step.poly_stream.data['ys'][idx], inverse=True)
    xs_dat, ys_dat = data_step.datProj(xs_map, ys_map, inverse=False)
    geometries = {
        "type": "Polygon",
        "coordinates": [
            list(map(list, zip(xs_dat, ys_dat)))
        ]
    }
    ta = affine.Affine(30.0, 0.0, float(data_step.da_bm['x'].min()),
                       0.0, -30.0, float(data_step.da_bm['y'].max()))
    poly_mask = shp2mask([geometries], data_step.da_bm,
                         transform=ta, outshape=data_step.da_bm.shape, default_value=1)
    print('clicked3')
    da_bm_tmp = data_step.da_bm.where(poly_mask == 1)
    print('clicked4')
    bm_hist_tmp = da_bm_tmp.hvplot.hist('Biomass', xlim=(0, 2000),
                                        bins=np.arange(0, 10000, 20))\
        .opts(height=200, width=300, fill_color=ps_c, fill_alpha=0.6,
              line_color='black', line_width=0.5, line_alpha=0.6,
              bgcolor=data_step.bg_col).options(toolbar=None)
    markdown = pn.pane.Markdown('## Region stats', height=50,
                                style={'font-family': "serif",
                                       'color': ps_c})
    print('clicked5')
    thresh_pct = round(float(da_bm_tmp.where(da_bm_tmp < 500).count())/
                       float(da_bm_tmp.count()) * 100, 0)
    thresh_text = pn.pane.Markdown(f'**{thresh_pct}%** of the region is estimated to have biomass ' +
                                   f'less than {500} kg/ha.',
                                   style={'font-family': "Helvetica"})
    thresh_list.append(pn.Column(pn.Row(pn.layout.HSpacer(), markdown, pn.layout.HSpacer()),
                                 bm_hist_tmp * hv.VLine(x=500).opts(line_color='black'),
                                 thresh_text,
                                 css_classes=['box1'], margin=5))
    #bm_gauge_obj = deepcopy(data_step.gauge_obj)
    #bm_gauge_obj['series'][0]['data'][0]['value'] = int(da_bm_tmp.mean().values)
    #bm_gauge_pane = pn.pane.ECharts(bm_gauge_obj, **data_step.gauge_opts)
    bm_list.append(pn.Column(pn.Row(pn.layout.HSpacer(), markdown, pn.layout.HSpacer()),
                             #bm_gauge_pane,
                             css_classes=['box1'], margin=5))
    #yr = int(data_step.da_bm_sel.YEAR.values)
    #ts_bm_yr_tmp = data_step.da_bm.where(poly_mask == 1).sel(date=slice(datetime(yr, 5, 1),
    #                                                                datetime(yr, 10, 31)))
    #ts_yr_list.append(ts_bm_yr_tmp.mean(dim=['x', 'y']).hvplot.line(x='date',
    #                                                               y='Biomass').opts(line_color=ps_c))
    #ts_bm_avg_tmp = data_step.da_bm.where(poly_mask == 1).groupby(da_bm.date.dt.dayofyear).mean(dim=['x', 'y'])
    #ts_avg_list.append(ts_bm_avg_tmp.hvplot.line(x='date', y='Biomass').opts(line_color=ps_c))
    da_cov_tmp = data_step.da_cov.where(poly_mask == 1)
    cov_factors = list(da_cov_tmp.type.values)
    cov_vals = [round(float(da_cov_tmp.sel(type=f).mean().values), 2) for f in cov_factors]
    from bokeh.models import NumeralTickFormatter
    pct_fmt = NumeralTickFormatter(format="0%")
    cov_colors = hv.Cycle(['red', 'green', 'blue'])
    cov_scatter_tmp = hv.Overlay([hv.Scatter(f) for f in list(zip(cov_factors, cov_vals))]) \
        .options({'Scatter': dict(xformatter=pct_fmt,
                                  size=15,
                                  fill_color=cov_colors,
                                  line_color=cov_colors,
                                  ylim=(0, 1))})
    cov_spike_tmp = hv.Overlay([hv.Spikes(f) for f in cov_scatter_tmp])\
        .options({'Spikes': dict(color=cov_colors, line_width=4,
                                 labelled=[], invert_axes=True, color_index=None,
                                 ylim=(0, 1))})
    cov_list.append(pn.Column(pn.Row(pn.layout.HSpacer(), markdown, pn.layout.HSpacer()),
                              (cov_spike_tmp * cov_scatter_tmp).options(height=200,
                                                                        width=300,
                                                                        bgcolor=data_step.bg_col,
                                                                        toolbar=None),
                              css_classes=['box1'], margin=5))

cov_list[0].servable()

In [ ]:
t0 = time.time()
#print(data_step.da_bm.sel(date=np.datetime64(data_step.date.value)).values)
print(data_step.da_bm.sel(date=np.datetime64(datetime(2021, 1, 18).date())).values)
#print(data_step.da_bm.isel(date=data_step.date.enabled_dates.index(data_step.date.value)).values)
#print(data_step.da_bm.isel(date=6).values)
t1 = time.time()
print(t1-t0)

In [ ]:
from skimage.draw import polygon
class getData(pm.Parameterized):
    bm_mod = pickle.load(open('src/models/CPER_HLS_to_VOR_biomass_model_lr_simp.pk', 'rb'))
    cov_mod = pickle.load(open('src/models/CPER_HLS_to_LPI_cover_pls_binned_model.pk', 'rb'))
    action = pm.Action(lambda x: x.param.trigger('action'), label='Load Data and Run Analysis')
    get_stats = pm.Action(lambda y: y.param.trigger('get_stats'), label='Calculate stats')
    username_input = pn.widgets.PasswordInput(name='NASA Earthdata Login', placeholder='Enter Username...')
    password_input = pn.widgets.PasswordInput(name='', placeholder='Enter Password...')
    year_picker = pn.widgets.Select(name='Year', options=[2021], value=2021)
    thresh_picker = pn.widgets.IntSlider(name='Threshold', start=200, end=2000, step=25, value=500,
                                 format=PrintfTickFormatter(format='%d kg/ha'))
    
    data_dict = {'date_range': [str(datetime(year_picker.value, 1, 1).date()),
                                str(datetime(year_picker.value, 12, 31).date())]}
 
    datCRS = crs.UTM(13)
    mapCRS = crs.GOOGLE_MERCATOR
    datProj = Proj(datCRS.proj4_init)
    mapProj = Proj(mapCRS.proj4_init)
    map_args = dict(crs=datCRS, rasterize=True, project=True, dynamic=True)
    base_opts = dict(projection=mapCRS, backend='bokeh', xticks=None, yticks=None, width=900, height=700,
                         padding=0, active_tools=['pan', 'wheel_zoom'], toolbar='left')
    map_opts = dict(projection=mapCRS, responsive=False, xticks=None, yticks=None, width=900, height=700,
                     padding=0, tools=['pan', 'wheel_zoom', 'box_zoom'],
                     active_tools=['pan', 'wheel_zoom'], toolbar='left')
    
    poly_opts = dict(fill_color=['', ''], fill_alpha=[0.0, 0.0], line_color=['#1b9e77', '#d95f02'],
                 line_width=[3, 3])    
    bg_col='#ffffff'

    css = '''
    .bk.box1 {
      background: #ffffff;
      border-radius: 5px;
      border: 1px black solid;
    }
    '''
    
    date = pn.widgets.DatePicker(name='Calendar', width=200)
      
    box_ctr_dat = [522435.0, 4519550.5]
    box_ctr_coord = datProj(box_ctr_dat[0], box_ctr_dat[1], inverse=True)
    box_ctr_map = mapProj(box_ctr_coord[0], box_ctr_coord[1], inverse=False)
    
    tiles = gv.tile_sources.EsriImagery.opts(**base_opts, level='glyph',
                                                                   xlim=(box_ctr_map[0] - 5000,
                                                                         box_ctr_map[0] + 5000),
                                                                   ylim=(box_ctr_map[1] - 5000,
                                                                         box_ctr_map[1] + 5000))
    labels = gv.tile_sources.EsriReference.opts(**base_opts, level='overlay')
       
    basemap = gv.tile_sources.EsriImagery.opts(projection=mapCRS, backend='bokeh', level='glyph')
    
    base_rng = hv.streams.RangeXY(source=tiles)
    
    polys = hv.Polygons([])

    poly_stream = streams.PolyDraw(source=polys, drag=True, num_objects=2,
                                    show_vertices=True, styles=poly_opts)
    edit_stream = streams.PolyEdit(source=polys, shared=True)
    
    gauge_obj = pn.indicators.Gauge(
        name='Biomass', bounds=(0, 2500), format='{value} kg/ha',
        colors=[(0.20, '#FF6E76'), (0.40, '#FDDD60'), (0.60, '#7CFFB2'), (1, '#58D9F9')],
        num_splits=5, value=0, align='center', title_size=12,
        start_angle=180, end_angle=0, height=200, width=300,
    )
    
    def __init__(self, **params):
        super(getData, self).__init__(**params)
        self.da = None
        self.da_cov = None
        self.da_bm = None
        self.da_bm_se = None
        self.da_thresh = None
        self.all_maps = None
        self.cov_map = None
        self.bm_map = None
        self.thresh_map = None
        
        self.cov_stats = None
        self.bm_stats = None
        self.thresh_stats = None
        self.stats_titles = None
                
        self.view = self._create_view()
        
    
    @pm.depends('action')
    def proc_data(self):
        message = 'Not yet launched'
        if self.username_input.value != '':
            message = 'username found'
            try:
                tmp_data = get_hls([self.username_input.value,self.password_input.value], 
                                   hls_data=self.data_dict, aws=True)
                #self.da = tmp_data.loc[dict(x=slice(517587.0, 527283.0), y=slice(4524402.0, 4514699.0))]
                message = self.base_rng.x_range
                coord_rng = self.mapProj(self.base_rng.x_range, self.base_rng.y_range, inverse=True)
                message = coord_rng
                dat_rng = np.round(self.datProj(coord_rng[0], coord_rng[1], inverse=False), 0)
                message = dat_rng
                self.da = tmp_data.loc[dict(x=slice(*dat_rng[0]), y=slice(*dat_rng[1][::-1]))]
                self.da['time'] = self.da.time.dt.floor("D")
                self.da = self.da.rename(dict(time='date'))
                da_mask = mask_hls(self.da['FMASK'])
                self.da = self.da.where(da_mask == 0)
                #self.da_bm = pred_bm(self.da, self.bm_mod)
                #self.da_bm = self.da_bm.where(self.da_bm > 0)
                #self.da_bm.name = 'Biomass'
                #self.da_cov = pred_cov(self.da, self.cov_mod)
                #self.da_cov = self.da_cov[['SD', 'GREEN', 'BARE']].to_array(dim='type')
                #self.da_cov = self.da_cov.where((self.da_cov < 1.0) | (self.da_cov.isnull()), 1.0)
                #self.da_cov.name = 'Cover'
                #self.da_bm_se = pred_bm_se(self.da, self.bm_mod)
                #self.da_bm_se = self.da_bm_se.where(self.da_bm.notnull())
                #self.da_thresh = pred_bm_thresh(self.da_bm, self.da_se, self.thresh_picker.value)
                #self.da_thresh.name = 'Threshold'
                self.date.enabled_dates = [pd.Timestamp(x).to_pydatetime().date() for x in self.da['date'].values]
                message = 'Success!' + str(datetime.now())
                return message
            except:
                return message + ': App Failure'
        else:
            return message
      
    @pm.depends('date.param')
    def create_maps(self):
        if self.da is not None:
            if self.date.value is not None:
                date_idx = self.date.enabled_dates.index(self.date.value)
                da_sel = self.da.isel(date=date_idx).persist()
            else:
                da_sel = self.da.isel(date=-1).persist()
            if self.edit_stream.data is not None:
                self.polys = self.edit_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
                                                    show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            elif self.poly_stream.data is not None:
                self.polys = self.poly_stream.element
                self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
                               show_vertices=True, styles=self.poly_opts)
                self.edit_stream = streams.PolyEdit(source=self.polys, shared=True)
            else:
                self.polys = self.polys
            da_bm = pred_bm(da_sel, self.bm_mod)
            da_bm = da_bm.where(da_bm > 0)
            da_bm.name = 'Biomass'
            da_cov = pred_cov(da_sel, self.cov_mod)
            da_cov = da_cov[['SD', 'GREEN', 'BARE']].to_array(dim='type')
            da_cov = da_cov.where((da_cov < 1.0) | (da_cov.isnull()), 1.0)
            da_cov.name = 'Cover'
            da_bm_se = pred_bm_se(da_sel, bm_mod)
            da_bm_se = da_bm_se.where(da_bm.notnull())
            da_thresh = pred_bm_thresh(da_bm, da_bm_se, self.thresh_picker.value)
            da_thresh.name = 'Threshold'
            
            #self.basemap = self.basemap.opts(xlim=(self.base_rng.x_range[0],
            #                                       self.base_rng.x_range[1]),
            #                                 ylim=(self.base_rng.y_range[0],
            #                                       self.base_rng.y_range[1]))
            self.cov_map = da_cov.hvplot.rgb(x='x', y='y', bands='type',
                                        **self.map_args).opts(**self.map_opts)
            self.bm_map = da_bm.hvplot(x='x', y='y',
                                  cmap='Viridis', clim=(100, 1000), colorbar=False,
                                  **self.map_args).opts(**self.map_opts)
            self.thresh_map = da_thresh.hvplot(x='x', y='y',
                                          cmap='YlOrRd', clim=(0.05, 0.95), colorbar=False,
                                          **self.map_args).opts(**self.map_opts)

            self.da_cov = da_cov
            self.da_bm = da_bm
            self.da_bm_se = da_bm_se
            self.da_thresh = da_thresh
            
            self.all_maps = pn.Tabs(#('Imagery', self.basemap * self.polys),
                           ('Cover', self.basemap * self.cov_map * self.polys),
                           ('Biomass', self.basemap * self.bm_map * self.polys),
                           ('Threshold', self.basemap * self.thresh_map * self.polys))
            
            #self.poly_stream = streams.PolyDraw(source=self.polys, drag=True, num_objects=2,
            #                        show_vertices=True, styles=self.poly_opts)
        
            return self.all_maps
        else:
            return pn.Column(self.tiles * self.labels)
        
    @pm.depends('get_stats')
    def show_stats(self):
        if self.poly_stream.data is not None:
            markdown_list = []
            thresh_list = []
            bm_list = []
            cov_list = []
            for idx, ps_c in enumerate(self.poly_stream.data['line_color']):
                r, c = polygon(data_step.poly_stream.data['xs'][idx]*-1, data_step.poly_stream.data['ys'][idx])
                r = r * -1.0
                bm_dat_tmp = self.bm_map[r][:,c].data
                bm_hist_tmp = bm_dat_tmp.hvplot.hist('Biomass', xlim=(0, 2000),
                                                    bins=np.arange(0, 10000, 20))\
                    .opts(height=200, width=300, fill_color=ps_c, fill_alpha=0.6,
                          line_color='black', line_width=0.5, line_alpha=0.6,
                          bgcolor=self.bg_col, align='center').options(toolbar=None)
                bm_gauge_obj = deepcopy(self.gauge_obj)
                bm_gauge_obj.value = int(bm_dat_tmp.mean()['Biomass'])
                bm_list.append(pn.Column(bm_gauge_obj,
                                         css_classes=['box1'], margin=5))
                markdown = pn.pane.Markdown('# Region ' + str(idx+1) + ' :', align='center',
                                            style={'font-family': "serif",
                                                   'color': ps_c})
                markdown_list.append(pn.Column(markdown, css_classes=['box1'], margin=5))
                thresh_pct = round(float(bm_dat_tmp.where(bm_dat_tmp < 500).count()['Biomass'])/
                                   float(bm_dat_tmp.count()['Biomass']) * 100, 0)
                thresh_text = pn.pane.Markdown(f'**{thresh_pct}%** of the region is estimated to have biomass ' +
                                               f'less than {500} kg/ha.',
                                               style={'font-family': "Helvetica"})
                thresh_list.append(pn.Column(bm_hist_tmp * hv.VLine(x=500).opts(line_color='black', height=200),
                                             thresh_text,
                                             css_classes=['box1'], margin=5, align='center'))
            self.stats_titles = pn.Column(*markdown_list)
            self.bm_stats = pn.Column(*bm_list)
            self.thresh_stats = pn.Column(*thresh_list)
            return pn.Row(self.stats_titles, self.bm_stats, self.thresh_stats)
        else:
            return pn.Row(None)
    def _create_view(self):
        layout = pn.Row(pn.Column(self.username_input,
                                  self.password_input,
                                  pn.Param(self.param,
                                           widgets={'action': pn.widgets.Button(name='Get data', width=200),
                                                    'get_stats': pn.widgets.Button(name='Calculate stats', width=200)},
                                  show_name=False),
                         self.proc_data,
                         self.date), pn.Column(self.create_maps,
                                               self.show_stats))
        return layout
                                        

In [3]:
import panel as pn
pn.extension()
pn.Row()

Row()

In [5]:
import holoviews as hv
hv.help(pn.Row)

Parameters of 'Row'

Parameters changed from their default values are marked in red.
Soft bound values are marked in cyan.
C/V= Constant/Variable, RO/RW = ReadOnly/ReadWrite, AN=Allow None

Name             Value        Type         Bounds     Mode  

align           'start'  ObjectSelector               V RW  
aspect_ratio      None     Parameter                V RW AN 
background        None     Parameter                V RW AN 
col_sizing        None     Parameter                V RW AN 
css_classes       None        List       (0, None)  V RW AN 
height            None      Integer      (0, None)  V RW AN 
height_policy    'auto'  ObjectSelector               V RW  
loading          False      Boolean        (0, 1)     V RW  
margin             0       Parameter                  V RW  
max_height        None      Integer      (0, None)  V RW AN 
max_width         None      Integer      (0, None)  V RW AN 
min_height        None      Integer      (0, None)  V RW AN 
min_width       